In [1]:
import sys
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import os
import pandas as pd
import numpy as np
import pickle as pickle

In [2]:
## functions

def load_pickled_in_filename(file):
    # load a simple pickled file and return it. its a bit different to the method used for the dictionary as this
    # is pure Python 3.x
    pickle_in = open(file, 'rb')
    return pickle.load(pickle_in)

In [3]:
oos_experiment_path = '/media/ak/My Passport/ExperimentData/AlternateLabelExperimentPath'
oos_experiment_symbol_folders = [f for f in os.listdir(oos_experiment_path) if str('pkl') not in f]

In [4]:
oos_experiment_symbol_folders

['AAL.L',
 'AV.L',
 'STAN.L',
 'TSCO.L',
 'ULVR.L',
 'VOD.L',
 'AZN.L',
 'GKN.L',
 'HSBA.L',
 'PRU.L',
 'PSON.L',
 'RB.L',
 'RDSb.L',
 'BATS.L',
 'BLT.L',
 'SHP.L',
 'RTO.L',
 'SDR.L',
 'DMGOa.L',
 'RSA.L',
 'SGE.L',
 'SMIN.L',
 'CEY.L',
 'CPG.L',
 'CCL.L',
 'ECM.L',
 'CPI.L']

In [5]:
oos_symbol_idx =0 # this corresponds to a symbol
symbol = oos_experiment_symbol_folders[oos_symbol_idx]
print(symbol)
oos_symbol_folder_files = [f for f in os.listdir(os.path.join(oos_experiment_path, symbol)) if str('_OOS_results') in f]

AAL.L


In [6]:
LabelsAlternateFive_symbol_files = [f for f in oos_symbol_folder_files if str('LabelsAlternateFive') in f]
LabelsAlternateFour_symbol_files = [f for f in oos_symbol_folder_files if str('LabelsAlternateFour') in f]
LabelsAlternateThree_symbol_files = [f for f in oos_symbol_folder_files if str('LabelsAlternateThree') in f]
LabelsAlternateTwo_symbol_files = [f for f in oos_symbol_folder_files if str('LabelsAlternateTwo') in f]
LabelsAlternateOne_symbol_files = [f for f in oos_symbol_folder_files if str('LabelsAlternateOne') in f]


In [7]:
label_list_input = LabelsAlternateOne_symbol_files
number_model_files = len(label_list_input)
number_model_files
label_string = label_list_input[0].split("_")[1]
label_string

'LabelsAlternateOne'

In [8]:
dict_list=[]
for pickle_file_idx in range(number_model_files):
    pickle_file_location = os.path.join(oos_experiment_path, symbol, LabelsAlternateOne_symbol_files[pickle_file_idx])
    df_w_Data = load_pickled_in_filename(pickle_file_location)
    ans = {label.split("_")[1] : {metric : np.mean([value[metric] for value in values.values()]) for metric in values[list(values)[0]].keys()} for label, values in df_w_Data.items()}
    dict_list.append(ans)


In [9]:
def concatenate_all_oos_per_label(dict_list):
    list_of_dfs =[pd.DataFrame(l) for l in dict_list]
    combined_results =pd.concat([pd.DataFrame(l) for l in dict_list], axis=1)
    combined_results
    return combined_results
    
    

In [10]:
df =concatenate_all_oos_per_label(dict_list)
# df =df.mean(axis=1)
# df = pd.DataFrame(df)
# df.rename(columns ={df.columns[0]: label_string}, inplace = True)

In [19]:
df

,20170117,20170707,20170726,20170822,20170119,20170123,20170124,20170127,20170130,20170131,...,20170804,20170807,20170809,20170814,20170817,20170818,20170821,20170823,20170824,20170830
Hamming Loss,0.660000,0.656923,0.716000,0.6775,0.664444,0.580000,0.579412,0.571515,0.671875,0.682903,...,0.639091,0.654,0.590000,0.61000,0.594286,0.678333,0.754,0.600000,0.590,0.61
accuracy,0.340000,0.343077,0.284000,0.3225,0.335556,0.420000,0.420588,0.428485,0.328125,0.317097,...,0.360909,0.346,0.410000,0.39000,0.405714,0.321667,0.246,0.400000,0.410,0.39
f1- macro,0.289730,0.268077,0.232667,0.2800,0.238056,0.197143,0.197353,0.200303,0.299063,0.273871,...,0.238182,0.278,0.193333,0.30625,0.301429,0.275000,0.202,0.190000,0.195,0.19
f1- micro,0.340000,0.343077,0.284000,0.3225,0.335556,0.420000,0.420588,0.428485,0.328125,0.317097,...,0.360909,0.346,0.410000,0.39000,0.405714,0.321667,0.246,0.400000,0.410,0.39
f1- weighted,0.317297,0.300385,0.223333,0.2975,0.245833,0.248571,0.249412,0.256667,0.329062,0.298065,...,0.246364,0.305,0.238889,0.35875,0.360000,0.280000,0.184,0.226667,0.235,0.22
precision,0.390000,0.379615,0.203333,0.3475,0.217778,0.177143,0.177647,0.184242,0.380312,0.373226,...,0.198182,0.381,0.166667,0.36500,0.374286,0.361667,0.206,0.156667,0.165,0.15
recall,0.340000,0.343077,0.284000,0.3225,0.335556,0.420000,0.420588,0.428485,0.328125,0.317097,...,0.360909,0.346,0.410000,0.39000,0.405714,0.321667,0.246,0.400000,0.410,0.39


In [12]:
def create_mean_list_w_correct_label(df, label_string):
    df = pd.DataFrame(df.mean(axis=1))
    df =  df.rename(columns ={df.columns[0]: label_string}, inplace = True)
    return df

In [21]:
create_mean_list_w_correct_label(df, label_string)
df_w_mean = pd.DataFrame(df.mean(axis =1))

In [22]:
print(df_w_mean.to_latex(index=True)) 

\begin{tabular}{lr}
\toprule
{} &         0 \\
\midrule
Hamming Loss &  0.648843 \\
accuracy     &  0.351157 \\
f1- macro    &  0.237748 \\
f1- micro    &  0.351157 \\
f1- weighted &  0.265395 \\
precision    &  0.267711 \\
recall       &  0.351157 \\
\bottomrule
\end{tabular}



In [14]:
df.columns[0]

'20170117'

In [15]:
 import numpy as np
ans = {label : {metric : np.mean([value[metric] for value in values.values()]) for metric in values[list(values)[0]].keys()} for label, values in df_w_Data.items()}

In [17]:
new_key = list(ans.keys())[0].split("_")[1]+'-' +list(ans.keys())[0].split("_")[2]

IndexError: list index out of range

In [18]:
key =list(df_test.keys())
"-".join((key[0].split("_")[1],key[0].split("_")[2]))

NameError: name 'df_test' is not defined

In [52]:
clean_dict = df_test[key[0]]
oos_dates =list(clean_dict.keys())
oos_date

'20170831'

In [41]:
data =clean_dict[oos_date]

In [54]:
test_df =pd.DataFrame.from_dict(data, orient='index')

In [56]:
test_df.columns.values

array([0])

In [59]:
test_df.rename(columns={0: str(oos_date)})

,20170831
accuracy,0.73
precision,0.99
recall,0.73
f1- weighted,0.84
f1- micro,0.73
f1- macro,0.29
Hamming Loss,0.27


In [143]:
results_path =os.listdir('/media/ak/My Passport/ExperimentData/Alt_Label_Results')

In [144]:
results_path

[]